In [69]:
import sys
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier 
from importlib.machinery import SourceFileLoader

# TODO Jupyter working directory is /notebooks. Therefore normal "from src.data" isn't working. Manually passing --notebook-dir
# when running the notebook didn't work. Needs to be investigated and fixed. This is a workaround
split_test_data = SourceFileLoader('split_test_data', '../src/data/split_test_data.py').load_module()
visualize = SourceFileLoader('visualize', '../src/visualization/visualize.py').load_module()

# True if you want to run with the Kaggle train and test set. Otherwise it will use split data for model optimization
use_kaggle_data = False 

Importing datasets for training (80%), testing (10%) and validation (10%)
and normalize features using MinMaxScaler. Load full Kaggle data and predict using Kaggle test set if training with that

In [70]:
if use_kaggle_data:
    X_train, y_train, X_test = split_test_data.load_kaggle_train_and_test_data('../data/raw/train.csv', '../data/raw/test.csv')
else:    
    X_train, y_train, X_test, y_test, X_valid, y_valid = \
    split_test_data.split_and_normalize('../data/raw/train.csv', '../data/processed')

X_train shape: (8000, 19)
y_train shape: (8000,)
X_test shape: (3799, 20)


Check details of the data if required

In [73]:
# X_train.describe()

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,0.540289,0.221109,0.193625,0.195862,0.193282,0.507819,0.487330,0.502123,0.481623,0.171917,0.175476,0.464686,0.195970,0.182269,0.188801,0.126915,0.180191,0.493076,0.222647
std,0.130678,0.126026,0.129303,0.132295,0.127564,0.134106,0.137176,0.136021,0.132696,0.114340,0.112825,0.057247,0.142849,0.128910,0.133664,0.105936,0.113229,0.022311,0.139090
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.450382,0.128350,0.098802,0.101562,0.099644,0.416122,0.392857,0.410256,0.388889,0.086420,0.090090,0.429748,0.090909,0.083333,0.089744,0.054688,0.083333,0.489130,0.115385
50%,0.541985,0.196051,0.164671,0.164062,0.163701,0.503268,0.500000,0.500000,0.480929,0.148148,0.153153,0.464874,0.163636,0.157407,0.160256,0.101562,0.166667,0.491848,0.192308
75%,0.625954,0.290550,0.260479,0.265625,0.259786,0.596950,0.571429,0.589744,0.572968,0.234568,0.234234,0.498353,0.272727,0.250000,0.256410,0.171875,0.250000,0.497283,0.288462
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Training the random forest

In [74]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state = 44, max_features="auto")
rf.fit(X_train, y_train.values.ravel()) # Converting column y values to 1d array

RandomForestClassifier(n_jobs=1, random_state=44)

Predicting using trained random forest 

In [75]:
pred = rf.predict(X_test.loc[:, 'GP':'TOV']) # Predicting using ranodm forest. Removing ID during predicting

result = pd.DataFrame(data = {'Id': X_test.loc[:,'Id'], 'TARGET_5Yrs': pred}) # Data frame with ID for csv writing
result_values = result[['TARGET_5Yrs']] # Extracting values for calculating stats

Saving the trainned model and writing result to a CSV file

In [76]:
joblib.dump(rf, "../models/random_forest_v1.joblib")
result.to_csv("../data/external/test_v3.csv", index = False)
result.shape

(3799, 2)

Show stats related to performance of the model if not using Kaggle dataset

In [78]:
if use_kaggle_data==False:
    visualize.show_random_forest_stats(result_values, y_test)
else:
    print("Kaggle dataset - No stats")

Kaggle dataset - No stats
